# ACS Data Cleaning and Exploration

## Importing Required Libraries

In [1]:
import os
import numpy as np
import pandas as pd
import pandas_gbq
from google.cloud import bigquery
from varname import nameof
%load_ext google.cloud.bigquery

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '../zori-data-extr-be793d5c3325.json'

# Set your default project here
pandas_gbq.context.project = 'bigquery-public-data'
pandas_gbq.context.dialect = 'standard'

## Using Google BigQuery to Download ACS Data

In [2]:
%%bigquery --use_rest_api ACS_2018
SELECT *
FROM `bigquery-public-data.census_bureau_acs.zip_codes_2018_5yr`

Downloading: 100%|██████████| 33120/33120 [00:05<00:00, 5694.27rows/s]


In [3]:
%%bigquery --use_rest_api ACS_2017
SELECT *
FROM `bigquery-public-data.census_bureau_acs.zip_codes_2017_5yr`

Downloading: 100%|██████████| 33120/33120 [00:09<00:00, 3658.63rows/s]


In [4]:
%%bigquery --use_rest_api ACS_2016
SELECT *
FROM `bigquery-public-data.census_bureau_acs.zip_codes_2016_5yr`

Downloading: 100%|██████████| 33120/33120 [00:06<00:00, 5361.62rows/s]


In [5]:
%%bigquery --use_rest_api ACS_2015
SELECT *
FROM `bigquery-public-data.census_bureau_acs.zip_codes_2015_5yr`

Downloading: 100%|██████████| 33120/33120 [00:10<00:00, 3151.05rows/s]


In [6]:
%%bigquery --use_rest_api ACS_2014
SELECT *
FROM `bigquery-public-data.census_bureau_acs.zip_codes_2014_5yr`

Downloading: 100%|██████████| 33120/33120 [00:05<00:00, 5996.65rows/s] 


## ACS Data Cleaning and Pre-Processing

In [7]:
pd.set_option('display.max_columns', None)
#Looking at the shape of each dataframe
print(ACS_2018.shape)
print(ACS_2017.shape)
print(ACS_2016.shape)
print(ACS_2015.shape)
print(ACS_2014.shape, '\n')

(33120, 240)
(33120, 252)
(33120, 252)
(33120, 247)
(33120, 252) 



Based on the data shown above, a number of columns are missing from the 2018 data set. Since the 2018 data set is the latest, all additional columns seen in prior year surveys will be dropped for consistency.

In [8]:
def clean_columns(df_year1, df_year2):
    """
    This function cleans the second dataframe to only include columns of the first dataframe.
    
    Arguments:
    
    df_year1 : (Pandas dataframe) dataframe containing the columns of interest
    df_year2 : (Pandas dataframe) dataframe containing the columns of interest + additional
    
    Returns
    df_year2 : (Pandas dataframe) modified version of the df_year2 dataframe from inputs
    
    """
    diff = np.setdiff1d(df_year2.columns, df_year1.columns)
    print(f'Columns from {nameof(df_year2)} non included in {nameof(df_year1)}:\n\nTotal of {len(diff)}\n\n{diff}\n\n')
    print('Removing columns...')
    
    #Dropping the columns from df_year2 that are not present in the df_year1
    df_year2 = df_year2.drop(columns = diff, inplace = True)
    
    print('Process complete\n\n')
    
    return df_year2

In [9]:
#Using the above defined function to clean columns
clean_columns(ACS_2018, ACS_2017)
clean_columns(ACS_2018, ACS_2016)
clean_columns(ACS_2018, ACS_2015)
clean_columns(ACS_2018, ACS_2014)

Columns from df_year2 non included in df_year1:

Total of 12

['amerindian_including_hispanic' 'asian_including_hispanic'
 'black_including_hispanic' 'commute_35_39_mins' 'commute_40_44_mins'
 'commute_5_9_mins' 'commute_60_89_mins' 'commute_90_more_mins'
 'households_retirement_income' 'male_60_61' 'male_62_64'
 'white_including_hispanic']


Removing columns...
Process complete


Columns from df_year2 non included in df_year1:

Total of 12

['amerindian_including_hispanic' 'asian_including_hispanic'
 'black_including_hispanic' 'commute_35_39_mins' 'commute_40_44_mins'
 'commute_5_9_mins' 'commute_60_89_mins' 'commute_90_more_mins'
 'households_retirement_income' 'male_60_61' 'male_62_64'
 'white_including_hispanic']


Removing columns...
Process complete


Columns from df_year2 non included in df_year1:

Total of 13

['amerindian_including_hispanic' 'asian_including_hispanic'
 'black_including_hispanic' 'commute_35_39_mins' 'commute_40_44_mins'
 'commute_5_9_mins' 'commute_60_89_mins'

In [10]:
#Looking at the shape of each dataframe to confirm column cleaning has been done appropriately
print(ACS_2018.shape)
print(ACS_2017.shape)
print(ACS_2016.shape)
print(ACS_2015.shape)
print(ACS_2014.shape, '\n')

(33120, 240)
(33120, 240)
(33120, 240)
(33120, 234)
(33120, 240) 



In [11]:
diff = np.setdiff1d(ACS_2018.columns, ACS_2015.columns)
diff

array(['pop_15_and_over', 'pop_divorced', 'pop_never_married',
       'pop_now_married', 'pop_separated', 'pop_widowed'], dtype=object)

As can be seen above for year 2015 there are less columns than in any other year. Before dropping the columns in all other survey years, looking at missingness of those 6 columns would be best.

In [206]:
#Adding a year column to each survey dataframe
ACS_2018['year'] = 2018
ACS_2017['year'] = 2017
ACS_2016['year'] = 2016
ACS_2015['year'] = 2015
ACS_2014['year'] = 2014

#Combining all ACS dataframes to one global ACS dataframe
ACS = pd.concat([ACS_2018, ACS_2017, ACS_2016, ACS_2015, ACS_2014], ignore_index = True)
ACS = ACS.reset_index().drop(columns = 'index')

### Keeping the Focus on Top 10 Zillow Metro Areas

The project is looking at predicting 2019 data as a base model (no Covid-19 effect) by using zip codes from the top 10 metro areas in the currently available ZRI index data. 

In [207]:
#Reading in the Focus Zipcode file
focus_zip = pd.read_csv('../Data/focus_zipcode.csv')

#Creating a list of pertinent zip codes in str format
focus_zip_l = focus_zip.RegionName.apply(lambda x: str(x)).to_list()

#Keeping rows that have focus zipcodes in created ACS dataframes
ACS = ACS.loc[ACS['geo_id'].isin(focus_zip_l)]

### Dropping Income Columns

Partly Parrots will be looking at IRS data for everything income related, as such all income related columns will be dropped.

In [208]:
ACS = ACS[ACS.columns.drop(list(ACS.filter(regex = 'income')))]

### Looking at Missingness

In [209]:
def missingness(df):
    """
    This function looks at the number of missing values in a dataframe
    
    Arguments:
    
    df          : (Pandas dataframe) dataframe of interest
    
    Returns
    
    missing_col : (dictionary) column name and number of missing values 
    
    """
    all_cols_miss = df.isna().mean().sort_values(ascending = False)

    missing_col_list = []
    missing_col = {}

    for i in range(all_cols_miss[all_cols_miss > 0].shape[0]):
        missing_col_list.append([all_cols_miss[all_cols_miss > 0].index[i], \
                                round(all_cols_miss[all_cols_miss > 0][i]*100, 2)])

    missing_col.update(missing_col_list)
    return missing_col

In [210]:
#Looking at missing values in data
missingness(ACS)

{'pop_never_married': 80.0,
 'pop_now_married': 80.0,
 'pop_separated': 80.0,
 'pop_widowed': 80.0,
 'pop_divorced': 80.0,
 'pop_15_and_over': 80.0,
 'pop_5_years_over': 60.0,
 'speak_spanish_at_home': 60.0,
 'speak_only_english_at_home': 60.0,
 'speak_spanish_at_home_low_english': 60.0,
 'aggregate_travel_time_to_work': 1.36,
 'median_rent': 0.46,
 'renter_occupied_housing_units_paying_cash_median_gross_rent': 0.4,
 'owner_occupied_housing_units_lower_value_quartile': 0.14,
 'median_year_structure_built': 0.1,
 'owner_occupied_housing_units_median_value': 0.07,
 'owner_occupied_housing_units_upper_value_quartile': 0.06}

##### Taking a closer look into the high missingness columns<br>
##### Marital Status

In [238]:
ACS.loc[ACS['pop_now_married'].isna()]

,geo_id,do_date,total_pop,households,median_age,white_pop,population_1_year_and_over,population_3_years_over,pop_5_years_over,pop_15_and_over,pop_16_over,pop_25_years_over,pop_25_64,pop_never_married,pop_now_married,pop_separated,pop_widowed,pop_divorced,not_us_citizen_pop,black_pop,asian_pop,hispanic_pop,amerindian_pop,other_race_pop,two_or_more_races_pop,hispanic_any_race,not_hispanic_pop,asian_male_45_54,asian_male_55_64,black_male_45_54,black_male_55_64,hispanic_male_45_54,hispanic_male_55_64,white_male_45_54,white_male_55_64,pop_determined_poverty_status,poverty,gini_index,housing_units,renter_occupied_housing_units_paying_cash_median_gross_rent,owner_occupied_housing_units_lower_value_quartile,owner_occupied_housing_units_median_value,owner_occupied_housing_units_upper_value_quartile,occupied_housing_units,housing_units_renter_occupied,vacant_housing_units,vacant_housing_units_for_rent,vacant_housing_units_for_sale,dwellings_1_units_detached,dwellings_1_units_attached,dwellings_2_units,dwellings_3_to_4_units,dwellings_5_to_9_units,dwellings_10_to_19_units,dwellings_20_to_49_units,dwellings_50_or_more_units,mobile_homes,housing_built_2005_or_later,housing_built_2000_to_2004,housing_built_1939_or_earlier,median_year_structure_built,married_households,nonfamily_households,family_households,households_public_asst_or_food_stamps,male_male_households,female_female_households,children,children_in_single_female_hh,median_rent,rent_burden_not_computed,rent_over_50_percent,rent_40_to_50_percent,rent_35_to_40_percent,rent_30_to_35_percent,rent_25_to_30_percent,rent_20_to_25_percent,rent_15_to_20_percent,rent_10_to_15_percent,rent_under_10_percent,owner_occupied_housing_units,million_dollar_housing_units,mortgaged_housing_units,different_house_year_ago_different_city,different_house_year_ago_same_city,families_with_young_children,two_parent_families_with_young_children,two_parents_in_labor_force_families_with_young_children,two_parents_father_in_labor_force_families_with_young_children,two_parents_mother_in_labor_force_families_with_young_children,two_parents_not_in_labor_force_families_with_young_children,one_parent_families_with_young_children,father_one_parent_families_with_young_children,father_in_labor_force_one_parent_families_with_young_children,commute_less_10_mins,commute_10_14_mins,commute_15_19_mins,commute_20_24_mins,commute_25_29_mins,commute_30_34_mins,commute_35_44_mins,commute_60_more_mins,commute_45_59_mins,commuters_16_over,walked_to_work,worked_at_home,no_car,no_cars,one_car,two_cars,three_cars,four_more_cars,aggregate_travel_time_to_work,commuters_by_public_transportation,commuters_by_bus,commuters_by_car_truck_van,commuters_by_carpool,commuters_by_subway_or_elevated,commuters_drove_alone,group_quarters,associates_degree,bachelors_degree,high_school_diploma,less_one_year_college,masters_degree,one_year_more_college,less_than_high_school_graduate,high_school_including_ged,bachelors_degree_2,bachelors_degree_or_higher_25_64,graduate_professional_degree,some_college_and_associates_degree,male_45_64_associates_degree,male_45_64_bachelors_degree,male_45_64_graduate_degree,male_45_64_less_than_9_grade,male_45_64_grade_9_12,male_45_64_high_school,male_45_64_some_college,male_45_to_64,employed_pop,unemployed_pop,pop_in_labor_force,not_in_labor_force,workers_16_and_over,armed_forces,civilian_labor_force,employed_agriculture_forestry_fishing_hunting_mining,employed_arts_entertainment_recreation_accommodation_food,employed_construction,employed_education_health_social,employed_finance_insurance_real_estate,employed_information,employed_manufacturing,employed_other_services_not_public_admin,employed_public_administration,employed_retail_trade,employed_science_management_admin_waste,employed_transportation_warehousing_utilities,employed_wholesale_trade,occupation_management_arts,occupation_natural_resources_construction_maintenance,occupation_production_transportation_material,occupation_sales_office,occupation_services,manag

Based on the results above, it seems like the data pertaining to marital status was either not reported by the individuals completing the survey or was not collected by the survey department. With an 80% reported missingness, with no way of comfortably imputing, the columns pertaining to marital status will be dropped.

In [240]:
#Dropping marital status columns
ACS = ACS.drop(columns = ['pop_never_married', 'pop_now_married', 'pop_separated', 'pop_widowed', 'pop_divorced'])

### Population Age Groups

To further reduce the number of columns in the ACS data set, columns reporting various population groups will be further grouped, with the total values removed from the data set.<br><br>
Before doing so, total values will be checked with the "dummified" groupings.

#### Checking Male Population Grouping and Totals

In [212]:
ACS['male_pop'] = ACS['male_pop'].astype(float)

#Creating a list of all columns containing the word 'male'
male_cols = [x for x in ACS.columns if x.startswith('male_')]

#Creating a list of male population grouped by age
male_ages = male_cols[1:male_cols.index('male_male_households')]

#Converting male population grouped by age columns to int 
ACS[male_ages] = ACS[male_ages].astype(int)
ACS['male_pop_check'] = ACS[male_ages].sum(axis = 1)
ACS['male_pop_check'] = ACS['male_pop_check'].astype(float)

#Comparing total male population from ACS with sum of male population grouped by age
print(f'Total rows in ACS: {ACS.shape[0]}')
print(f'Total rows with unmatched male population: {ACS.loc[ACS1.male_pop != ACS.male_pop_check].shape[0]}\n\n')

#Creating an 'unknown' male age column to later drop the total male population column
ACS['male_x_age'] = ACS['male_pop'] - ACS['male_pop_check']
ACS['male_x_age'] = ACS['male_x_age'].astype(float)

#Reporting percentage of males that haven't reported their age
perc_male_x_age = round(ACS['male_x_age'].sum() / ACS['male_pop'].sum() *100,2)
print(f'Total percentage of males that didn\'t have an age group: {perc_male_x_age}%')

Total rows in ACS: 11165
Total rows with unmatched male population: 11164


Total percentage of males that didn't report their age: 5.24%


Age groups will further be combined into the following:<br>
- Males under 18
- Males from 18 to 24
- Males from 25 to 49
- Males from 50 to 66
- Males above 67

In [216]:
#Combining age groups
ACS['male_under_18'] = ACS['male_under_5'] + ACS['male_5_to_9'] + ACS['male_10_to_14'] + ACS['male_15_to_17']
ACS['male_18_to_24'] = ACS['male_18_to_19'] + ACS['male_20'] + ACS['male_21'] + ACS['male_22_to_24']
ACS['male_25_to_49'] = ACS['male_25_to_29'] + ACS['male_30_to_34'] + ACS['male_35_to_39'] + ACS['male_40_to_44'] +\
                        ACS['male_45_to_49']
ACS['male_50_to_66'] = ACS['male_50_to_54'] + ACS['male_55_to_59'] + ACS['male_65_to_66']
ACS['male_above_67'] = ACS['male_67_to_69'] + ACS['male_70_to_74'] + ACS['male_75_to_79'] + ACS['male_80_to_84'] +\
                        ACS['male_85_and_over']

In [218]:
#Dropping all other columns related to male population by age group
ACS = ACS.drop(columns = male_ages)
ACS = ACS.drop(columns = ['male_pop_check', 'male_pop'])

#### Checking Female Population Grouping and Totals

In [230]:
ACS['female_pop'] = ACS['female_pop'].astype(float)

#Creating a list of all columns containing the word 'male'
female_cols = [x for x in ACS.columns if x.startswith('female_')]

#Creating a list of male population grouped by age
female_ages = female_cols[1:female_cols.index('female_female_households')]

#Converting male population grouped by age columns to int 
ACS[female_ages] = ACS[female_ages].astype(float)
ACS['female_pop_check'] = ACS[female_ages].sum(axis = 1)
ACS['female_pop_check'] = ACS['female_pop_check'].astype(float)

#Comparing total male population from ACS with sum of male population grouped by age
print(f'Total rows in ACS: {ACS.shape[0]}')
print(f'Total rows with unmatched female population: {ACS.loc[ACS.female_pop != ACS.female_pop_check].shape[0]}\n\n')

#Creating an 'unknown' female age column to later drop the total male population column
ACS['female_x_age'] = ACS['female_pop'] - ACS['female_pop_check']
ACS['female_x_age'] = ACS['female_x_age'].astype(float)

#Reporting percentage of males that haven't reported their age
perc_female_x_age = round(ACS['female_x_age'].sum() / ACS['female_pop'].sum() *100,2)
print(f'Total percentage of females that didn\'t have an age group: {perc_female_x_age}%')

Total rows in ACS: 11165
Total rows with unmatched female population: 0


Total percentage of females that didn't have an age group: 0.0%


Seems like women always have a reported age! Very different from the male population.<br><br>
Age groups will further be combined into the following:<br>
- Females under 18
- Females from 18 to 24
- Females from 25 to 49
- Females from 50 to 66
- Females above 67

In [233]:
#Combining age groups
ACS['female_under_18'] = ACS['female_under_5'] + ACS['female_5_to_9'] + ACS['female_10_to_14'] + ACS['female_15_to_17']
ACS['female_18_to_24'] = ACS['female_18_to_19'] + ACS['female_20'] + ACS['female_21'] + ACS['female_22_to_24']
ACS['female_25_to_49'] = ACS['female_25_to_29'] + ACS['female_30_to_34'] + ACS['female_35_to_39'] + ACS['female_40_to_44'] +\
                        ACS['female_45_to_49']
ACS['female_50_to_66'] = ACS['female_50_to_54'] + ACS['female_55_to_59'] + ACS['female_65_to_66']
ACS['female_above_67'] = ACS['female_67_to_69'] + ACS['female_70_to_74'] + ACS['female_75_to_79'] + ACS['female_80_to_84'] +\
                        ACS['female_85_and_over']

In [234]:
#Dropping all other columns related to male population by age group
ACS = ACS.drop(columns = female_ages)
ACS = ACS.drop(columns = 'female_pop')